In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import backtrader as bt
from backtrader.sizers import PercentSizer
import backtrader.analyzers as btanalyzers

from termcolor import colored

from base import StrategyBase
import util

%matplotlib inline
%matplotlib widget

In [10]:
PATH = r'../data/stooq/data/daily/'
MARKET = r'de/xetra stocks'
TICKER = 'bayn'
FILE_CSV = PATH+MARKET+"/1/"+TICKER+".csv"

# APPLE_FILE_CSV = r'../data/stooq/data/daily/us/nasdaq stocks/1/aapl.csv'
# APPLE_FILE_CSV = r'../data/stooq/data/daily/de/xetra stocks/1/bayn.csv'
print(FILE_CSV)

../data/stooq/data/daily/de/xetra stocks/1/bayn.csv


In [11]:
class FullMoney(PercentSizer):
    params = (
        ('percents', 99),
    )

In [12]:
class StochRSI(bt.Indicator):
    lines = ('fastk', 'fastd',)

    params = (
        ('k_period', 3),
        ('d_period', 3),
        ('period', 14),
        ('stoch_period', 14),
        ('upperband', 70.0),
        ('lowerband', 30.0),
    )

    def __init__(self, base_indicator):
        rsi_ll = bt.ind.Lowest(base_indicator, period=self.p.period)
        rsi_hh = bt.ind.Highest(base_indicator, period=self.p.period)
        stochrsi = (base_indicator - rsi_ll) / ((rsi_hh - rsi_ll) + 0.00001)

        self.l.fastk = k = bt.indicators.MovingAverageSimple(100.0 * stochrsi, period=self.p.k_period)
        self.l.fastd = bt.indicators.MovingAverageSimple(k, period=self.p.d_period)

In [13]:
class BasicRSI(StrategyBase):
    params = dict(
        period_ema_fast=10,
        period_ema_slow=100
    )

    def __init__(self):
        StrategyBase.__init__(self)
        self.log("Using RSI/EMA strategy")

        self.ema_fast = bt.indicators.EMA(period=self.p.period_ema_fast)
        self.ema_slow = bt.indicators.EMA(period=self.p.period_ema_slow)
        self.rsi = bt.indicators.RelativeStrengthIndex()

        self.profit = 0

    def update_indicators(self):
        self.profit = 0
        if self.buy_price_close and self.buy_price_close > 0:
            self.profit = float(self.data0.close[0] - self.buy_price_close) / self.buy_price_close

    def next(self):
        self.update_indicators()

        if self.order:  # waiting for pending order
            return

        # stop Loss
        if self.profit < -0.03:
            self.log("STOP LOSS: percentage %.3f %%" % self.profit)
            self.short()

        if self.last_operation != "BUY":
            if self.rsi < 30 and self.ema_fast > self.ema_slow:
                self.long()

        if self.last_operation != "SELL":
            if self.rsi > 70:
                self.short()

In [14]:
class SimpleRSI(StrategyBase):
    def __init__(self):
        StrategyBase.__init__(self)
        self.log("Using RSI strategy")

        self.rsi = bt.indicators.RelativeStrengthIndex()

        self.profit = 0

    def update_indicators(self):
        self.profit = 0
        if self.buy_price_close and self.buy_price_close > 0:
            self.profit = float(self.data0.close[0] - self.buy_price_close) / self.buy_price_close

    def next(self):
        self.update_indicators()

        if self.order:  # waiting for pending order
            return

        if self.last_operation != "BUY":
            if self.rsi < 30:
                self.long()

        if self.last_operation != "SELL":
            if self.rsi > 70:
                self.short()

In [15]:
#Get Data
data1 = bt.feeds.GenericCSVData(dataname=FILE_CSV,params = (
        ('nullvalue', float('NaN')),
        ('dtformat', '%Y%m%d'),
        ('timeframe', bt.TimeFrame.Days),
        ('fromdate', datetime(2000, 1, 1)),
        ('todate', datetime(2000, 12, 31)),
        ('compression', 1),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3,
        ('close', 4),
        ('volume', 5)
        )))

cerebro = bt.Cerebro(quicknotify=True)

cerebro.adddata(data1)

broker = cerebro.getbroker()
broker.setcommission(commission=0.001)  # Simulating exchange fee
broker.setcash(100000.0)
cerebro.addsizer(FullMoney)

# Analyzers to evaluate trades and strategies
# SQN = Average( profit / risk ) / StdDev( profit / risk ) x SquareRoot( number of trades )
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharperatio')
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual')
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(btanalyzers.DrawDown)

# Include Strategy
cerebro.addstrategy(SimpleRSI)

# Starting backtrader bot
initial_value = cerebro.broker.getvalue()
print('Starting Portfolio Value: %.2f' % initial_value)
results = cerebro.run()
strat = results[0]

portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

# Print analyzers - results
final_value = cerebro.broker.getvalue()
print('Final Portfolio Value: %.2f' % final_value)
print('Profit %.3f%%' % ((final_value - initial_value) / initial_value * 100))
util.print_trade_analysis(results[0].analyzers.ta.get_analysis())
util.print_sqn(results[0].analyzers.sqn.get_analysis())

# If no name has been specified, the name is the class name lowercased
tret_analyzer = strat.analyzers.getbyname('timereturn')
maxdrawdown = strat.analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = strat.analyzers.returns.get_analysis()['rnorm100']
sharpe = strat.analyzers.sharperatio.get_analysis()['sharperatio']
print(f"Max Drawdown: {maxdrawdown:.2f}%\nCAGR: {cagr:.2f}%\nSharpe: {sharpe:.3f}")

Starting Portfolio Value: 100000.00
[09-04-21 15:21] Base strategy initialized
[09-04-21 15:21] Using RSI strategy
[05-02-10 23:59] Buy ordered: $30.89
[08-02-10 23:59] BUY EXECUTED, Price: 30.99, Cost: 99296.03, Comm 99.30
[10-09-10 23:59] Sell ordered: $34.73
[13-09-10 23:59] SELL EXECUTED, Price: 35.04, Cost: 99296.03, Comm 112.30
[13-09-10 23:59] OPERATION PROFIT, GROSS 13000.70, NET 12789.10
[16-03-11 23:59] Buy ordered: $34.79
[17-03-11 23:59] BUY EXECUTED, Price: 34.96, Cost: 112228.58, Comm 112.23
[02-05-11 23:59] Sell ordered: $40.80
[03-05-11 23:59] SELL EXECUTED, Price: 40.71, Cost: 112228.58, Comm 130.67
[03-05-11 23:59] OPERATION PROFIT, GROSS 18445.18, NET 18202.28
[04-08-11 23:59] Buy ordered: $35.55
[05-08-11 23:59] BUY EXECUTED, Price: 34.62, Cost: 126301.81, Comm 126.30
[03-01-12 23:59] Sell ordered: $36.23
[04-01-12 23:59] SELL EXECUTED, Price: 35.98, Cost: 126301.81, Comm 131.24
[04-01-12 23:59] OPERATION PROFIT, GROSS 4941.46, NET 4683.92
[21-08-15 23:59] Buy order

In [16]:
import matplotlib
matplotlib.use('agg')

%matplotlib inline
%matplotlib widget

cerebro.plot(iplot= False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[<Figure size 432x288 with 5 Axes>]]